# Análise e Controle de Sistemas Lineares Incertos por meio de Desigualdades Matriciais Lineares: Exercício

## Descrição

Eeste notebook explora conceitos avançados de análise e controle de sistemas lineares incertos através da formulação e resolução de problemas de programação linear utilizando as bibliotecas CVX e Mosek em Python, com ênfase em Inequações Matriciais Lineares (LMIs).

### Data de Criação

2024-01-21

### Data de Modificação

2024-01-22

## Autor

Andevaldo da Encarnação Vitório

## Importação das Bibliotecas Necessárias

In [ ]:
import cvxpy as cp
import numpy as np
import math

In [359]:
def show_matrix(tag, matrix, decimal_places=2):
    """
    Apresenta uma matriz com a quantidade de casas decimais desejadas.

    Parâmetros:
    - matrix: numpy.ndarray, a matriz a ser apresentada.
    - casas_decimais: int, o número de casas decimais desejadas (padrão é 2).
    """
    pattern = "{:." + str(decimal_places) + "f}"

    def format_elem(elem):
        return pattern.format(elem)
    
    width = [max(map(len, map(format_elem, coluna))) for coluna in matrix.T]

    print(tag, "=")

    nspaces = sum(width) + 2 * matrix.shape[1]

    print("    ┌" + " " * nspaces + "┐")
    for line in matrix:
        formatted_line = "  ".join(format_elem(e).rjust(largura) for e, largura in zip(line, width))
        print("    │ " + formatted_line + " │")
    print("    └" + " " * nspaces + "┘")
    print()


## Problema 1

### Descrição

Considere o seguinte problema de otimização convexo baseado em desigualdades matriciais lineares:

$$ \underset{P}{\min} \space Tr(P) $$
$$ P > 0, \space\space A^TP+PA+Q < 0 $$

Resolva com

$$
A = \begin{bmatrix}
0 & 1 \\
-2 & -3
\end{bmatrix}
, \space
Q = 
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$


In [372]:
# Problema 1 - Parte 1

# Definição da Variável P
P = cp.Variable((2,2), symmetric=True)

# Definiçaõ dos Parâmetros
A = cp.Parameter((2,2), value=[[0., -2.],[1, -3]])
Q = cp.Parameter((2,2), value=np.eye(2))

# Apresentação dos parâmetros
show_matrix('A', A.value)
show_matrix('Q', Q.value)


# Definição do problema de otimização convexa: objetivo e restrições
obj = cp.Minimize(cp.trace(P))
constraints = [P >> 0, A.T @ P + P @ A + Q << 0]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação do resultado obtido na resolução
show_matrix('P', P.value)

A =
    ┌              ┐
    │  0.00   1.00 │
    │ -2.00  -3.00 │
    └              ┘

Q =
    ┌            ┐
    │ 1.00  0.00 │
    │ 0.00  1.00 │
    └            ┘


Valor ótimo: 1.4999999999962756 

P =
    ┌            ┐
    │ 1.25  0.25 │
    │ 0.25  0.25 │
    └            ┘



Resolva agora com o seguinte problema de otimização alternativo:

$$ \underset{P,\space \rho}{\min}\space Tr(\rho) $$
$$ P > 0, \space\space A^TP + PA + Q < 0, \space\space \rho \geq Tr(P) $$

In [374]:
# Problema 1 - Parte 2

# Definição da Variáveis P e rho
P = cp.Variable((2,2), symmetric=True)
ρ = cp.Variable()

# Definiçaõ dos Parâmetros
A = cp.Parameter((2,2), value=[[0., -2.],[1., -3.]])
Q = cp.Parameter((2,2), value=np.eye(2))

# Apresentação dos parâmetros
show_matrix('A', A.value)
show_matrix('Q', Q.value)

# Definição do problema de otimização convexa: objetivo e restrições
obj = cp.Minimize(ρ)
constraints = [P >> 0, A.T @ P + P @ A + Q << 0, ρ >= cp.trace(P)]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação do resultado obtido na resolução
show_matrix('P', P.value)
print("ρ = ", ρ.value)

A =
    ┌              ┐
    │  0.00   1.00 │
    │ -2.00  -3.00 │
    └              ┘

Q =
    ┌            ┐
    │ 1.00  0.00 │
    │ 0.00  1.00 │
    └            ┘


Valor ótimo: 1.4999999999962756 

P =
    ┌            ┐
    │ 1.25  0.25 │
    │ 0.25  0.25 │
    └            ┘

ρ =  1.4999999999962756


## Problema 2

### Descrição

Para ilustrar o cômputo da norma $\mathscr{H}_2 = \lVert H(s) \rVert_2$ usando LMIs, considere o sistema massa-mola apresentado em (Iwasaki, 1996), ilustrado na figura abaixo:

<br>
<div style="text-align:center;">
  <picture>
    <source media="(prefers-color-scheme: light)" srcset="assets/imgs/sistema_massa_mola_light.png">
    <source media="(prefers-color-scheme: dark)" srcset="assets/imgs/sistema_massa_mola_dark.png">
  
  <img src="assets/imgs/sistema_massa_mola_dark.png" alt="Sistema Massa Mola" width="60%"   title="Sistema Massa Mola">
  </picture>
</div>
<br>

A mesma função de transferência de (Iwasaki, 1996) é considerada, isto é, da força de entrada $d$ aplicada à massa $m_1$ para o sinal de erro $e = x_2$ (posição da massa $m_2$). O espaço de estado deste sistema é:

<br>

$$
\begin{bmatrix}
\dot{x_1} \\ \dot{x_2} \\ \ddot{x_1} \\ \ddot{x_2}
\end{bmatrix}

= 

\begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\displaystyle-\frac{k_1 + k_2}{m_1} & \displaystyle\frac{k_2}{m_1} &\displaystyle-\frac{c_0}{m_1} & 0 \\
\displaystyle\frac{k_2}{m_2} &\displaystyle-\frac{k_2}{m_2} &0 &\displaystyle-\frac{c_0}{m_2}
\end{bmatrix}

\begin{bmatrix}
x_1 \\ x_2 \\ \dot{x_1} \\ \dot{x_2}
\end{bmatrix}

+

\begin{bmatrix}
0 \\
0 \\
\displaystyle \frac{1}{m_1} \\
0 \\
\end{bmatrix}

d
$$

<br>

$$
Y =
\begin{bmatrix}
0 & 1 & 0 & 0
\end{bmatrix}

\begin{bmatrix}
x_1 \\ x_2 \\ \dot{x_1} \\ \dot{x_2}
\end{bmatrix}
$$

<br>

Portanto, as matrizes $A$, $B$ e $C$ são dadas por

<br>

$$
A = \begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\displaystyle-\frac{k_1 + k_2}{m_1} & \displaystyle\frac{k_2}{m_1} &\displaystyle-\frac{c_0}{m_1} & 0 \\
\displaystyle\frac{k_2}{m_2} &\displaystyle-\frac{k_2}{m_2} &0 &\displaystyle-\frac{c_0}{m_2}
\end{bmatrix}
, \space
B =
\begin{bmatrix}
0 \\
0 \\
\displaystyle \frac{1}{m_1} \\
0 \\
\end{bmatrix}
, \space
C =
\begin{bmatrix}
0 & 1 & 0 & 0
\end{bmatrix}
, \space
D = 0
$$

<br>

Os valores dos parâmetros são: massas $m_1 = 1$ e $m_2 = 0.5$; elasticidade das molas $k_1 = k_2 = 1$ e as forças de atrito $f_1$ e $f_2$ dadas por $f_i = −c_0 \cdot \dot{x_i}, \space (i = 1, 2)$, sendo que
$c_0 = 2$ é o coeficiente de atrito viscoso.

Determine a norma $\mathscr{H} _2$ deste sistema usando o seguinte problema de otimização:

$$ \underset{\rho, \space W = W^T > 0}{\min} \space\space \rho $$

sujeito a

$$
rho \geq Tr(CWC^T), \space
AW + WA^T + BB^T < 0
$$

Na solução ótima $\rho = \rho^\star$, tem se

$$ \lVert H(s) \rVert ^ 2 _ 2 = \rho^\star $$

onde, $H(s)$ é a matriz de transferência do sistema, dada por:

$$ H(s) = C(sI- A)^{-1}B +D  $$

In [377]:
# Problema 2

# Definição dos Parâmetros
m1 = 1.
m2 = 0.5
k1 = 1.
k2 = 1.
c0 = 2.

A = cp.Parameter((4, 4), value=[[0., 0., - ((k1 + k2) / m1), (k2 / m2)],
                               [0., 0., (k2 / m1), - (k2 / m2)],
                               [1., 0., - (c0 / m1), 0],
                               [0., 1., 0., -(c0 / m2)]])
B = cp.Parameter((4, 1), value=[[0., 0., 1. / m1, 0.]])
C = cp.Parameter((1, 4), value=[[0], [1], [0], [0]])

print('Parâmetros:\n')
show_matrix('A', A.value)
show_matrix('B', B.value)
show_matrix('C', C.value)

# Definição das Variáveis
ρ = cp.Variable()
W = cp.Variable((4, 4), symmetric=True)

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(ρ)
constraints = [W >> 0]
constraints += [ρ >= cp.trace(C @ W @ C.T)]
constraints += [A @ W + W @ A.T + B @ B.T << 0]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação dos Resultados
show_matrix('W', W.value)
print("ρ = ", ρ.value, '\n')
print("Norma H2 = ", math.sqrt(ρ.value), '\n')

Parâmetros:

A =
    ┌                            ┐
    │  0.00   0.00   1.00   0.00 │
    │  0.00   0.00   0.00   1.00 │
    │ -2.00   1.00  -2.00   0.00 │
    │  2.00  -2.00   0.00  -4.00 │
    └                            ┘

B =
    ┌      ┐
    │ 0.00 │
    │ 0.00 │
    │ 1.00 │
    │ 0.00 │
    └      ┘

C =
    ┌                        ┐
    │ 0.00  1.00  0.00  0.00 │
    └                        ┘


Valor ótimo: 0.0925925924451939 

W =
    ┌                            ┐
    │  0.16   0.08  -0.00   0.04 │
    │  0.08   0.09  -0.04  -0.00 │
    │ -0.00  -0.04   0.23   0.00 │
    │  0.04  -0.00   0.00   0.02 │
    └                            ┘

ρ =  0.0925925924451939 

Norma H2 =  0.30429030948289154 



## Problema 3

### Descrição

Determine a norma $\mathscr{H}_{\infty}$ do sistema massa-mola usando o seguinte problema de otimização:

$$ \underset{P = P^T > 0}{\min} \space \space \mu $$

sujeito a

$$
\begin{bmatrix}
A^TP + PA + C^TC & PB + C^TD\\
B^TP + D^TC & D^TD - \mu
\end{bmatrix} 
< 0
$$

No ótimo $\mu = \mu^{\star}$, a norma $\mathscr{H}_{\infty}$ é dada por

$$ \lVert H(s) \rVert _ \infty = \sqrt{\mu ^ \star}$$

In [380]:
# Problema 3

# Definição dos Parâmetros
m1 = 1.
m2 = 0.5
k1 = 1.
k2 = 1.
c0 = 2.

A = cp.Parameter((4, 4), value=[[0., 0., - ((k1 + k2) / m1), (k2 / m2)],
                                [0., 0., (k2 / m1), - (k2 / m2)],
                                [1., 0., - (c0 / m1), 0],
                                [0., 1., 0., -(c0 / m2)]])
B = cp.Parameter((4, 1), value=[[0., 0., 1. / m1, 0.]])
C = cp.Parameter((1, 4), value=[[0], [1], [0], [0]])
D = cp.Parameter((1, 1), value=[[0]])
I = cp.Parameter((1, 1), value=np.identity(1))

print('Parâmetros:\n')
show_matrix('A', A.value)
show_matrix('B', B.value)
show_matrix('C', C.value)
show_matrix('D', D.value)

# Definição das Variáveis
P = cp.Variable((4, 4), symmetric=True)
µ = cp.Variable()
M = cp.bmat([[A.T @ P + P @ A + C.T @ C, P @ B + C.T @ D], 
            [B.T @ P + D.T @ C, D.T @ D - µ * I]])

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(µ)
constraints = [P >> 0, M << 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação dos Resultados
show_matrix('P', P.value)
print("µ = ", µ.value, '\n')
print("Norma Hinf = ", math.sqrt(µ.value), '\n')

Parâmetros:

A =
    ┌                            ┐
    │  0.00   0.00   1.00   0.00 │
    │  0.00   0.00   0.00   1.00 │
    │ -2.00   1.00  -2.00   0.00 │
    │  2.00  -2.00   0.00  -4.00 │
    └                            ┘

B =
    ┌      ┐
    │ 0.00 │
    │ 0.00 │
    │ 1.00 │
    │ 0.00 │
    └      ┘

C =
    ┌                        ┐
    │ 0.00  1.00  0.00  0.00 │
    └                        ┘

D =
    ┌      ┐
    │ 0.00 │
    └      ┘


Valor ótimo: 0.9999999978766486 

P =
    ┌                          ┐
    │  2.02  -0.02  0.50  0.20 │
    │ -0.02   4.02  0.50  0.80 │
    │  0.50   0.50  0.94  0.16 │
    │  0.20   0.80  0.16  0.64 │
    └                          ┘

µ =  0.9999999978766486 

Norma Hinf =  0.9999999989383243 



## Exercício 4

Considere um modelo de satélite (Gahinet et al., 1995) que consiste em dois corpos rı́gidos (corpo principal e de instrumentação) conectados por um ligação flexı́vel modelada como uma mola com constante de torque k e amortecimento viscoso $f$. A Figura 2 apresenta uma ilustração do satélite, sendo $\theta_1$ e $\theta_2$ os ângulos de guinada para o corpo principal e para o módulo de instrumentação, respectivamente.

<br>

$$
\begin{bmatrix}
\dot{\theta_1} \\ \dot{\theta_2} \\ \ddot{\theta_1} \\ \ddot{\theta_2}
\end{bmatrix}

= 

\begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
-k & k & -f & f \\
k & -k & f & -f
\end{bmatrix}

\begin{bmatrix}
\theta_1 \\ \theta_2 \\ \dot{\theta_1} \\ \dot{\theta_2}
\end{bmatrix}

+

\begin{bmatrix}
0 \\
0 \\
0 \\
1 \\
\end{bmatrix}

w

+

\begin{bmatrix}
0 \\
0 \\
0 \\
1 \\
\end{bmatrix}

T

$$

<br>

$$
y =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{bmatrix}

\begin{bmatrix}
\theta_1 \\ \theta_2 \\ \dot{\theta_1} \\ \dot{\theta_2}
\end{bmatrix}

+ 

\begin{bmatrix}
0 \\
0 \\
1 \\
\end{bmatrix}

T
$$

<br>

sendo T o torque de controle e w uma perturbação sobre o corpo principal.

$$ \underset{\rho, \space X = X^T, \space Z, \space W = W^T > 0}{\min} \space \space \rho $$

sujeito a

<br>

$$
\begin{bmatrix}
X & CW + DZ  \\
WC^T + Z^TD^T & W 
\end{bmatrix} 
> 0

, \space

\rho \geq Tr(X)

, \space

\begin{bmatrix}
AW + B_2Z + WA^T + Z^TB^T_2 & B_1 \\
B_1^T & -I 
\end{bmatrix} 
< 0
$$

<br>

com $K = ZW^{-1}$ e $\lVert H(s) \rVert ^2_2 = \rho^\star$.

In [370]:
# Problema 3

# Definição dos Parâmetros
k = 0.2
f = 0.0039
J1 = 1.
J2 = 1.

A = cp.Parameter((4, 4), value=[[0., 0., -k, k],
                                [0., 0., k, -k],
                                [1., 0., -f, f],
                                [0., 1., f, -f]])


B1 = cp.Parameter((4, 1), value=[[0., 0., 0., 1. / J1]])
B2 = cp.Parameter((4, 1), value=[[0., 0., 0., 1. / J1]])
C = cp.Parameter((3, 4), value=[[1, 0, 0],
                                [0, 1, 0],
                                [0, 0, 0],
                                [0, 0, 0]])

D = cp.Parameter((3, 1), value=[[0, 0, 1]])

I = cp.Parameter((1, 1), value=np.identity(1))

print('Parâmetros:\n')
show_matrix('A', A.value)
show_matrix('B1', B1.value)
show_matrix('B2', B2.value)
show_matrix('C', C.value)
show_matrix('D', D.value)


# Definição das Variáveis
ρ = cp.Variable()
X = cp.Variable((4, 4), symmetric=True)
Z = cp.Variable((1, 4))
W = cp.Variable((4, 4), symmetric=True)

zeros = np.zeros((1, 4))

M12 = cp.vstack([C @ W + D @ Z, zeros])
M21 = cp.hstack([W @ C.T + Z.T @ D.T, zeros.T])

M = cp.bmat([[X, M12],
              [M21, W]])

N = cp.bmat([[A @ W + B2 @ Z + W @ A.T + Z.T @ B2.T, B1],
             [B1.T, -I]])

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(ρ)
constraints = [W >> 0]
constraints += [M >> 0]
constraints += [ρ >= cp.trace(X)]
constraints += [N << 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("Valor ótimo:", prob.value, '\n')

K = np.dot(Z.value, np.linalg.inv(W.value))

# Apresentação dos Resultados
print("ρ = ", ρ.value, '\n')
show_matrix("X", X.value)
show_matrix("Z", Z.value)
show_matrix("W", W.value)
show_matrix("K", K)

print("Norma H2 = ", math.sqrt(ρ.value), '\n')



Parâmetros:

A =
    ┌                            ┐
    │  0.00   0.00   1.00   0.00 │
    │  0.00   0.00   0.00   1.00 │
    │ -0.20   0.20  -0.00   0.00 │
    │  0.20  -0.20   0.00  -0.00 │
    └                            ┘

B1 =
    ┌      ┐
    │ 0.00 │
    │ 0.00 │
    │ 0.00 │
    │ 1.00 │
    └      ┘

B2 =
    ┌      ┐
    │ 0.00 │
    │ 0.00 │
    │ 0.00 │
    │ 1.00 │
    └      ┘

C =
    ┌                        ┐
    │ 1.00  0.00  0.00  0.00 │
    │ 0.00  1.00  0.00  0.00 │
    │ 0.00  0.00  0.00  0.00 │
    └                        ┘

D =
    ┌      ┐
    │ 0.00 │
    │ 0.00 │
    │ 1.00 │
    └      ┘

Valor ótimo: 1.6596603573653155 

ρ =  1.6596603573653155 

X =
    ┌                            ┐
    │  0.17  -0.01   0.01  -0.00 │
    │ -0.01   0.23  -0.32  -0.00 │
    │  0.01  -0.32   1.27  -0.00 │
    │ -0.00  -0.00  -0.00   0.00 │
    └                            ┘

Z =
    ┌                           ┐
    │ 0.01  -0.32  -0.00  -0.50 │
    └                      